In [15]:
def box_contains_centroid(box, centroid):
    return centroid[0] >= box[0] and centroid[0] <= box[2] and centroid[1] >= box[1] and centroid[1] <= box[3]

def get_centroid_of_box(box):
    return box[0]+(box[2]-box[0])/2, box[1]+(box[3]-box[1])/2

def bb_iou(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [6]:
import os
import sys

from PIL import Image
import PIL.ImageDraw as ImageDraw
import numpy as np
import time

indystudy_path = '/home/sean/Documents/indystudy/'

sys.path.append(indystudy_path)

from keras_yolo3.yolo import YOLO

annotation_path = os.path.join(indystudy_path, 'data', 'train.txt')
with open(annotation_path) as f:
    lines = f.readlines()
      
detector = YOLO()

window_size = (416, 416)

#for line in lines:
line = lines[0]
line = line.split()
image = Image.open(line[0])
iw, ih = image.size
h, w = window_size
gt_boxes = np.array([np.array(list(map(int,box.split(',')))) for box in line[1:]])

draw = ImageDraw.Draw(image)

/home/sean/Documents/indystudy/data/trained_weights_final.h5 model, anchors, and classes loaded.


In [7]:
predicted_boxes = []
for y in range(0, ih, h):
    for x in range(0, iw, w):
        area = (x, y, x+w, y+h)
        window = image.crop(area)

        # predicted bounding boxes
        _, window_predictions = detector.detect_image_silent(window)

        for bb in window_predictions:
            adjusted_bb = [bb['left'] + x, bb['top'] + y, bb['right'] + x, bb['bottom'] + y]
            
            predicted_boxes.append(adjusted_bb)
            
            draw.rectangle(adjusted_bb, outline="blue")

In [19]:
# true bounding boxes
false_negatives = 0
false_positives = 0
true_positives = 0
chopped_detections = 0

used_prediction_indices = {}
for bb in gt_boxes:
    
    preds_for_gt = []
    
    for i in range(len(predicted_boxes)):
        if not used_prediction_indices.get(i, False) and box_contains_centroid(bb, get_centroid_of_box(predicted_boxes[i])):
            preds_for_gt.append(predicted_boxes[i])
            
            # mark prediction as used
            used_prediction_indices[i] = True

    if len(preds_for_gt) == 0: # nothing detected for gt
        false_negatives += 1

        draw.rectangle([bb[0], bb[1], bb[2], bb[3]], outline="red")
    elif len(preds_for_gt) > 1: # multiple boxes for one gt
        chopped_detections += 1

        draw.rectangle([bb[0], bb[1], bb[2], bb[3]], outline="yellow")
    else:
        if bb_iou([bb[0], bb[1], bb[2], bb[3]], preds_for_gt[0]) > 0.5:
            true_positives += 1
            draw.rectangle([bb[0], bb[1], bb[2], bb[3]], outline="lime")
        else:
            false_negatives += 1
            draw.rectangle([bb[0], bb[1], bb[2], bb[3]], outline="teal")

false_positives = len([i for i in range(len(predicted_boxes)) if not used_prediction_indices.get(i, False)])

precision = true_positives / (true_positives+false_positives)
recall = true_positives / (true_positives+false_negatives)

print(" False Negatives:", false_negatives, "\n",
      "Chopped Detections:", chopped_detections, "\n",
      "True Positives:", true_positives, "\n",
      "False Positives:", false_positives, "\n",
      "Precision:", precision, "\n",
      "Recall:", recall)
    
image.save('first_train_example_detections_with_gt.tiff')
#break

 False Negatives: 156 
 Chopped Detections: 361 
 True Positives: 1931 
 False Positives: 32 
 Precision: 0.9836984207845135 
 Recall: 0.9252515572592238
